In [17]:
%matplotlib inline
import json
import pandas as pd
import urllib.request
from pandas.io.json import json_normalize

ModuleNotFoundError: No module named 'urllib.urlretrieve'


Computing a connectome with sparse inverse covariance
=======================================================

This example constructs a functional connectome using the sparse inverse
covariance.

We use the `MSDL atlas
<https://team.inria.fr/parietal/18-2/spatial_patterns/spatial-patterns-in-resting-state/>`_
of functional regions in rest, and the
:class:`nilearn.input_data.NiftiMapsMasker` to extract time series.

Note that the inverse covariance (or precision) contains values that can
be linked to *negated* partial correlations, so we negated it for
display.

As the MSDL atlas comes with (x, y, z) MNI coordinates for the different
regions, we can visualize the matrix as a graph of interaction in a
brain. To avoid having too dense a graph, we represent only the 20% edges
with the highest values.




Retrieve the atlas and the data
--------------------------------



In [1]:
from nilearn import datasets
atlas = datasets.fetch_atlas_msdl()
# Loading atlas image stored in 'maps'
atlas_filename = atlas['maps']
# Loading atlas data stored in 'labels'
labels = atlas['labels']

# Loading the functional datasets
data = datasets.fetch_adhd(n_subjects=1)

# TODO(nina): Load data from api



# print basic information on the dataset
print('First subject functional nifti images (4D) are at: %s' %
      data.func[0])  # 4D data

/usr/local/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



Dataset created in /Users/nina/nilearn_data/msdl_atlas



Downloaded 209734 of 209734 bytes (100.0%,    0.0s remaining) ...done. (2 seconds, 0 min)
Extracting data from /Users/nina/nilearn_data/msdl_atlas/5d25e157f36214b8ca9a12fd417aac1c/MSDL_rois.zip..... done.
/usr/local/lib/python3.6/site-packages/numpy/lib/npyio.py:2266: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)



Dataset created in /Users/nina/nilearn_data/adhd



Downloaded 3072 of 3072 bytes (100.0%,    0.0s remaining) ...done. (0 seconds, 0 min)
Extracting data from /Users/nina/nilearn_data/adhd/74a52d3552145e11365bb68453d1a70c/adhd40_metadata.tgz..... done.
/usr/local/lib/python3.6/site-packages/nilearn/datasets/func.py:501: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  dtype=None)


Downloaded 44414948 of 44414948 bytes (100.0%,    0.0s remaining) ...done. (47 seconds, 0 min)
Extracting data from /Users/nina/nilearn_data/adhd/26dc434877313cabfa02a4e93691edaf/adhd40_0010042.tgz...

First subject functional nifti images (4D) are at: /Users/nina/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz


.. done.


In [35]:
# TODO(nina): extract nii of matching tasks id

url_root = 'https://neurovault.org/api/images/'

page = 100
dfs = []
nii_files = []
nii_data = []
while True:
    page_url = url_root + '?offset=%d' % page

    with urllib.request.urlopen(page_url) as url:
        data = json.loads(url.read().decode())
        results = pandas.io.json.json_normalize(data['results'])
        #nii_file = urllib.parse.quote(str(results['file']))
        nii_files = results['file']
        nii_ids = results['id']
 
        for one_nii, one_id in zip(nii_files, nii_ids):
            urllib.request.urlretrieve(one_nii, '/Users/nina/code/mental_tasks/data/{}.nii.gz'.format(one_id))
        
        if data['next'] is 'null':
            break
        else:
            page += 100
        if page > 200:
            break
            
    

Extract time series
--------------------



In [ ]:
from nilearn.input_data import NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True,
                         memory='nilearn_cache', verbose=5)

time_series = masker.fit_transform(data.func[0],
                                   confounds=data.confounds)

Compute the sparse inverse covariance
--------------------------------------



In [ ]:
from sklearn.covariance import GraphLassoCV
estimator = GraphLassoCV()

estimator.fit(time_series)

Display the connectome matrix
------------------------------



In [ ]:
from nilearn import plotting
# Display the covariance

# The covariance can be found at estimator.covariance_
plotting.plot_matrix(estimator.covariance_, labels=labels,
                     figure=(9, 7), vmax=1, vmin=-1,
                     title='Covariance')

And now display the corresponding graph
----------------------------------------



In [ ]:
coords = atlas.region_coords

plotting.plot_connectome(estimator.covariance_, coords,
                         title='Covariance')

Display the sparse inverse covariance
--------------------------------------
we negate it to get partial correlations



In [ ]:
plotting.plot_matrix(-estimator.precision_, labels=labels,
                     figure=(9, 7), vmax=1, vmin=-1,
                     title='Sparse inverse covariance')

And now display the corresponding graph
----------------------------------------



In [ ]:
plotting.plot_connectome(-estimator.precision_, coords,
                         title='Sparse inverse covariance')

plotting.show()